In [1]:
%load_ext autoreload
%autoreload 2
from models import NNDynamicModel, MPCcontroller
import tensorflow as tf
import numpy as np
import json
import pickle

/Users/loulinhui/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
DYNAMIC_MODEL_PARAMS = {
    'l2_regularizer_scale': 1e-5,
    'activation': 'tanh',
    'output_activation': None,
    'learning_rate': 0.005,
    'batch_size': 128,
    'n_layers': 5,
    'size': 128,
    'iterations': 40,
}

In [3]:
# load data
with open('./gao/data_buffer.pkl', 'rb') as f:
    data = pickle.load(f)

data = data.sample(data._tail)

In [4]:
# [obs, action, nobs, r]
obs = [d for d in data if d[0][1] < 0]
obs = np.array(obs)
print(obs[2])

[list([0.0, -50.0, 105.0, 51.0, 6.2920000000000975])
 list([2.0, 0.37846549794879236])
 list([0.0, 439.0, 105.0, 51.0, 6.320000000000107]) 1]


In [5]:
obs = [d[0] for d in data]
normalization = [np.min(obs, axis=0), np.max(obs, axis=0), np.mean(obs, axis=0)]
print(normalization)

session = tf.Session()

[array([  0.   , -50.   ,  90.   ,  17.   ,   6.153]), array([  0.   , 625.   , 105.   ,  51.   ,   6.743]), array([  0.        , 450.741625  ,  96.84375   ,  29.41579167,
         6.22045446])]


In [6]:
model = NNDynamicModel(
    name='test',
    sess=session,
    normalization=normalization,
    **DYNAMIC_MODEL_PARAMS
)

In [7]:
init = tf.global_variables_initializer()
session.run(init)

In [8]:
model.fit(data)

on iter_step 0, loss = 0.6132059, max_loss = 0.8509675, min_loss = 0.4855779
on iter_step 1, loss = 0.5569801, max_loss = 0.6771551, min_loss = 0.4506190
on iter_step 2, loss = 0.5422396, max_loss = 0.7704645, min_loss = 0.4062078
on iter_step 3, loss = 0.5321300, max_loss = 0.7087847, min_loss = 0.4229354
on iter_step 4, loss = 0.5321613, max_loss = 0.6532180, min_loss = 0.4211412
on iter_step 5, loss = 0.5197561, max_loss = 0.6300232, min_loss = 0.4096825
on iter_step 6, loss = 0.5166356, max_loss = 0.6428299, min_loss = 0.3724757
on iter_step 7, loss = 0.5156615, max_loss = 0.6747386, min_loss = 0.3939956
on iter_step 8, loss = 0.5188743, max_loss = 0.6391256, min_loss = 0.3718780
on iter_step 9, loss = 0.5133657, max_loss = 0.6635657, min_loss = 0.3770050
on iter_step 10, loss = 0.5198277, max_loss = 0.6506032, min_loss = 0.4060705
on iter_step 11, loss = 0.5125126, max_loss = 0.6602467, min_loss = 0.4025634
on iter_step 12, loss = 0.5087498, max_loss = 0.6625913, min_loss = 0.4114

In [ ]:
mpc = MPCcontroller(dyn_model=model) 

In [ ]:
nd = [d for d in data if d[-1] > 0]
obs, action, nobs, reward = nd[2]
mreward = model.predict([obs], [action])
print(mreward)

In [ ]:
from game_controller import GameEnv

In [ ]:
env = GameEnv()
env.start()

obs = None
step = 1000
while step > 0:
    if env.status == 'CRASHED':
        env.restart()
    if obs is None:
        obs = env.get_observation()
    if obs is not None:
        action = mpc.get_action(obs)
        obs, _, _ = env.perform_action(action)
        step -= 1